In [1]:
from env_creator import qsimpy_env_creator
import os
import csv
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np
from typing import List, Dict

In [2]:
import random

class AcoSolutions:
    def __init__(self, env, num_ants=10, num_iterations=100, evaporation_rate=0.5, pheromone_init=1.0):
        """
        Initializes the ACO algorithm parameters.
        Args:
            - env: The environment on which the ACO will operate.
            - num_ants (int): Number of ants used in the algorithm.
            - num_iterations (int): Total iterations to run the algorithm.
            - evaporation_rate (float): Rate at which pheromone decays.
            - pheromone_init (float): Initial pheromone level.
        """
        self.env = env
        self.num_ants = num_ants
        self.num_iterations = num_iterations
        self.evaporation_rate = evaporation_rate
        self.pheromone_init = pheromone_init
        
        # Initialize pheromone levels and solution storage
        self.pheromones = {}  # Dictionary to store pheromone levels for each node
        self.best_solution = None
        self.best_cost = float("inf")

    def initialize_pheromones(self):
        """
        Initialize pheromone levels for each node with the initial pheromone level.
        """
        for node in self.env.nodes:
            self.pheromones[node] = self.pheromone_init

    def run(self):
        """
        Runs the ACO algorithm for a set number of iterations.
        """
        self.initialize_pheromones()
        self.results = []
        # Reset the subset of QTasks 
        self.env.round = 1

        for iteration in range(self.num_iterations):
            # Initialize the temporary array to store the results of the QTasks execution for each episode
            arr_temp = {
                "total_completion_time": 0.0,
                "rescheduling_count": 0.0
            }
            terminated = False

            # Reset the environment and setup the quantum resources
            self.env.reset()
            self.env.setup_quantum_resources()
            self.rr_index = 0

            while not terminated:
                # Get the action with the given control
                action = self.aco()
        
                self.env.reset()
                self.env.setup_quantum_resources()
                action = self.random()
                obs, reward, terminated, done, info = self.env.step(action)
                
                
                self.env.qsp_env.run()
    
    def aco(self):
        """
        Runs the ACO algorithm for a set number of iterations.
        """
        self.initialize_pheromones()
        
        for iteration in range(self.num_iterations):
            solutions = []
            for _ in range(self.num_ants):
                solution, cost = self.construct_solution()
                solutions.append((solution, cost))
                if cost < self.best_cost:
                    self.best_solution = solution
                    self.best_cost = cost
            self.update_pheromones(solutions)

    def construct_solution(self):
        """
        Constructs a solution for a single ant based on pheromone levels and completion times.
        Returns:
            - solution (list): The sequence of nodes visited by the ant.
            - cost (float): The total completion time associated with this path.
        """
        path = []
        total_time = 0
        
        # Start from a random node
        unvisited_nodes = set(self.env.qnodes)
        current_node = random.choice(list(unvisited_nodes))
        unvisited_nodes.remove(current_node)
        path.append(current_node)
        
        while unvisited_nodes:
            next_node = self.choose_next_node(unvisited_nodes)
            path.append(next_node)
            total_time += self.env.get_completion_time(next_node)
            current_node = next_node
            unvisited_nodes.remove(current_node)

        return path, total_time

    def choose_next_node(self, unvisited_nodes):
        """
        Chooses the next node for an ant to visit based on pheromone levels and completion times.
        Args:
            - current_node: The current node where the ant is located.
            - unvisited_nodes: Set of nodes not yet visited by the ant.
        Returns:
            - next_node: The chosen next node.
        """
        probabilities = []
        
        for node in unvisited_nodes:
            alpha = 
            beta = 
            pheromone = self.pheromones[node] ** self.alpha
            heuristic = (1 / self.env.get_completion_time(node)) ** self.beta  # Inverse of completion time as heuristic
            probabilities.append((node, pheromone * heuristic))
        
        # Normalize probabilities
        total_probability = sum(prob[1] for prob in probabilities)
        probabilities = [(node, prob / total_probability) for node, prob in probabilities]

        # Roulette-wheel selection for next node
        choice = random.uniform(0, 1)
        cumulative_probability = 0
        for node, probability in probabilities:
            cumulative_probability += probability
            if choice <= cumulative_probability:
                return node

    def update_pheromones(self, solutions):
        """
        Updates pheromone levels on nodes visited by ants, applying evaporation and reinforcement.
        Args:
            - solutions (list): List of tuples with (path, cost) for each ant's solution.
        """
        # Evaporate pheromones
        for node in self.pheromones:
            self.pheromones[node] *= (1 - self.evaporation_rate)

        # Add pheromones based on solutions
        for path, cost in solutions:
            for node in path:
                self.pheromones[node] += 1.0 / cost

    def get_best_solution(self):
        """
        Returns the best solution found by the algorithm.
        """
        return self.best_solution, self.best_cost


    def _plot_results(self, paths) -> None:
        """
        Plot the results of the episodes.
        """
        for path in paths:
            df1 = pd.read_csv(path['path'])

            plt.plot(df1['Episode'], df1['Total Completion Time'], ".-", color=path['color'], label=path['label'])

            self._summarize_results(df1, path['label'])
        
        plt.ylabel('Total Completion Time')
        plt.xlabel('Evaluation Episode')
        plt.legend(loc=2)
        plt.gca().xaxis.set_major_locator(mticker.MultipleLocator(10))
        plt.show()

    def _summarize_results(self, values, label) -> None:
        """
        Summarize the results of the episodes.
        """
        print("Results Summary for" + label + "solution:")
        print(f"Number of Episodes: {self.num_episodes}")
        print(f"Total Completion Time: {sum(values['Total Completion Time'])}")
        print(f"Average Rescheduling Count: {sum(values['Rescheduling Count']) / self.num_episodes}")

In [3]:
class HeuristicSolutions:
    def __init__(self, env, num_episodes=100):

        # Initialize the environment
        self.env = env
        self.num_episodes = num_episodes

        # Initialize the results of heuristic solutions
        self.results = []
        # Round Robin index for the QNodes. Example: [0, 1, 2, 3, 4, 0, 1, 2, 3, 4, ...]
        self.rr_index = 0
        # Priority index of Greedy solution after sorting the QNodes based on the waiting time
        self.greedy_index = 0

    def run(self, control):
        """
        Run the heuristic solutions for the given algorithm (control).
        Args:
            - control (str): The heuristic algorithm to use. Options: "greedy", "random", "round_robin", "greedy_error"
        """

        self.results = []
        # Reset the subset of QTasks 
        self.env.round = 1

        for _ in range(self.num_episodes):

            # Initialize the temporary array to store the results of the QTasks execution for each episode
            arr_temp = {
                "total_completion_time": 0.0,
                "rescheduling_count": 0.0
            }
            terminated = False

            # Reset the environment and setup the quantum resources
            self.env.reset()
            self.env.setup_quantum_resources()
            self.rr_index = 0

            while not terminated:
                # Get the action with the given control
                if control == "greedy":
                    action = self.greedy(self.greedy_index)
                elif control == "random":
                    action = self.random()
                elif control == "round_robin":
                    action = self.round_robin()
                elif control == "greedy_error":
                    action = self.greedy_error(self.greedy_index)
                
                obs, reward, terminated, done, info = self.env.step(action)
                
                # If the QNode is busy or not satisfied, move to the next priority QNode
                self.greedy_index += 1
                if reward > 0:
                    """Get the results of the QTask execution

                    Values:
                        - Total Completion Time: waiting_time + execution_time
                        - Rescheduling Count: rescheduling_count
                    """
                    # Reset priority index of Greedy solution if QTasks are satisfied
                    self.greedy_index = 0

                    arr_temp["total_completion_time"] += info["scheduled_qtask"].waiting_time + info["scheduled_qtask"].execution_time
                    arr_temp["rescheduling_count"] += info["scheduled_qtask"].rescheduling_count
            self.env.qsp_env.run()
            # Final results of the episode
            self.results.append(arr_temp)

        # Save the results to a CSV file
        self._save_to_csv(control)
                
    def greedy(self, greedy_index):
        # Sort the QNodes based on the next available time (or waiting time) and select the QNode with the smallest waiting time
        greedy_strategy = sorted(self.env.qnodes, key=lambda x: x.next_available_time)
        return self.env.qnodes.index(greedy_strategy[greedy_index])

    def random(self):
        # Randomly select a QNode
        action = self.env.action_space.sample()
        return action
    
    def round_robin(self):
        # Select the QNode based on the Round Robin index
        action = self.rr_index % self.env.n_qnodes
        self.rr_index += 1
        return action
    
    def greedy_error(self, greedy_index, g_error="Readout_assignment_error"):
        # Sort the QNodes based on the next available time (or waiting time) and select the QNode with the 
        # smallest waiting time and smallest error (default is readout_error) in the qnode
    
        greedy_strategy = sorted(self.env.qnodes, key=lambda x: (x.next_available_time, x.error[g_error]))
        return self.env.qnodes.index(greedy_strategy[greedy_index])

    def _save_to_csv(self, control) -> None:
        """
        Save values and episodes to a CSV file.
        """

        file_name = "./results/heuristics2/" 

        if not os.path.exists(file_name):
            os.makedirs(file_name)

        file_name += control + ".csv"
        # Open the CSV file in write mode
        with open(file_name, mode='w', newline='') as file:
            writer = csv.writer(file)
            
            # Write the header
            writer.writerow(['Episode', 'Total Completion Time', 'Rescheduling Count'])
            
            # Write the data
            for i in range(len(self.results)):
                writer.writerow([i, self.results[i]['total_completion_time'], self.results[i]['rescheduling_count']])
        print("CSV file saved to " + file_name)

    def _plot_results(self, paths) -> None:
        """
        Plot the results of the episodes.
        """
        for path in paths:
            df1 = pd.read_csv(path['path'])

            plt.plot(df1['Episode'], df1['Total Completion Time'], ".-", color=path['color'], label=path['label'])

            self._summarize_results(df1, path['label'])
        
        plt.ylabel('Total Completion Time')
        plt.xlabel('Evaluation Episode')
        plt.legend(loc=2)
        plt.gca().xaxis.set_major_locator(mticker.MultipleLocator(10))
        plt.show()

    def _summarize_results(self, values, label) -> None:
        """
        Summarize the results of the episodes.
        """
        print("Results Summary for" + label + "solution:")
        print(f"Number of Episodes: {self.num_episodes}")
        print(f"Total Completion Time: {sum(values['Total Completion Time'])}")
        print(f"Average Rescheduling Count: {sum(values['Rescheduling Count']) / self.num_episodes}")

In [4]:
if __name__ == "__main__":

    # Create the QSimPy environment
    env_config={
                "obs_filter": "rescale_-1_1",
                "reward_filter": None,
                "dataset": "qdataset/qsimpyds_100_sub_24.csv",
            }

    env = qsimpy_env_creator(env_config)

    # Run the heuristic solutions
    # heuristics = HeuristicSolutions(env, num_episodes=100)
    # heuristics.run("greedy")
    # heuristics.run("random")
    # heuristics.run("round_robin")
    # heuristics.run("greedy_error")

    #Run the ACO 
    aco = ACOSolution(env, num_episodes=10, num_ants=3, num_iterations=5)
    aco.run()

    # Plot the results
    paths = [
        # {
        #     "label": "random",
        #     "path": "./results/heuristics2/random.csv",
        #     "color": "red"
        # },
        # {
        #     "label": "round robin",
        #     "path": "./results/heuristics2/round_robin.csv",
        #     "color": "blue"
        # },
        # {
        #     "label": "greedy",
        #     "path": "./results/heuristics2/greedy.csv",
        #     "color": "black"
        # },
        # {
        #     "label": "greedy_error",
        #     "path": "./results/heuristics2/greedy_error.csv",
        #     "color": "green"
        # },
        {
           "label": "ACO", 
           "path": "./results/heuristics2/aco.csv", 
           "color": "purple"
        }

    ]
    #heuristics._plot_results(paths)
    aco._plot_results(paths)
    

c:\Users\hp\Music\Qintern\research\qsimpy\qenv\lib\site-packages\gymnasium\spaces\box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


KeyboardInterrupt: 